In [1]:
import os

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin = DIRECTORY_URL + name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

811008/807992 [==============================] - 0s 0us/step


'/home/tanmay/.keras/datasets'

Iterate through the files, loading each one into its own dataset. <br/>

Each example needs to be individually labeled, so use tf.data.Dataset.map to apply a labeler function to each one. <br/>
This will iterate over every example in the dataset, returning (example, label) pairs.

In [3]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [4]:
# Params

BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [5]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration = False)

In [6]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Achilles in his turn threw, and struck the round shield of Aeneas at'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Attends thee; I am Hermes, by command'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Then thus, divinely fair, with tears she said.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Quick pantings heaved his chest, copious the sweat'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'house of Hades."'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)


In [ ]:
# Tokenize

tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

In [ ]:
# Encode examples
# Create an encoder by passing the vocabulary_set to tfds.features.text.TokenTextEncoder. 
# The encoder's encode method takes in a string of text and returns a list of integers.

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

You want to use Dataset.map to apply this function to each element of the dataset. <br/>
Dataset.map runs in graph mode.<br/>
Graph tensors do not have a value.<br/>
In graph mode you can only use TensorFlow Ops and functions.<br/>
So you can't .map this function directly.<br/>
You need to wrap it in a tf.py_function. The tf.py_function will pass regular tensors (with a value and a .numpy() method to access it), to the wrapped python function.

In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp = [text, label], 
                                       Tout = (tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually
    
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes = ([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes = ([None],[]))

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]